In [1]:
%matplotlib inline
import os.path
from joblib import load, dump
from nilearn.plotting import view_img_on_surf, view_img
import pandas as pd
import numpy as np

/home/brain/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [7]:
def load_scores(id_layer):
    title = "conv{}_scores.pkl"
    filename = title.format(id_layer)
    scores = load(filename)
    return scores
    

In [8]:
def verify_integrity_scores(id_layer) : 
    scores = load_scores(id_layer)
    for n_neurons, neuron_scores in scores.items():
        subject_list = []
        for id_subject, score_vector in neuron_scores.items():
            subject_list.append(id_subject)
        if len(subject_list) < 16 :
            print("Warning with {} neurons, layer {}".format(n_neurons, id_layer))
        else : 
            print("Layer {} for {} neurons has all the subjects".format(id_layer, n_neurons))

In [9]:
def build_df():
    df = pd.DataFrame()
    for id_layer in range(5,8): 
        layer_scores = load_scores(id_layer)
        index=['id_layer','id_subject', 'n_neurons', 'n_fold','r2_max']
        for n_neurons, neuron_scores in layer_scores.items():
            for id_subject, scores in neuron_scores.items():
                for fold, score in enumerate(scores) :
                    new_row = pd.Series([id_layer, id_subject, n_neurons, fold+1, score], index = index)
                    df = df.append(new_row, ignore_index=True)
    return df


In [10]:
df = build_df()

FileNotFoundError: [Errno 2] No such file or directory: 'conv5_scores.pkl'

In [54]:
for id_layer in range(6,8) :
    verify_integrity_scores(id_layer)

Layer 6 for 1000 neurons has all the subjects
Layer 7 for 1000 neurons has all the subjects


In [211]:
### Generate mean table 
table1_mean = pd.DataFrame()
index = ['50','100','500','1000','1500']
for id_layer in range(1,8):
    layer = df[df.id_layer == id_layer]
    f = layer.groupby('n_neurons').mean()['r2_max']
    table1_mean = table1_mean.append(f,ignore_index = True)
table1_mean.to_pickle("table1_mean.pkl")
print(table1_mean.to_latex(float_format="{:0.2f}".format))

\begin{tabular}{lrrrrr}
\toprule
{} &  50.0   &  100.0  &  500.0  &  1000.0 &  1500.0 \\
\midrule
0 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
1 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
2 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
3 &    0.00 &    0.01 &    0.02 &    0.03 &    0.03 \\
4 &    0.11 &    0.13 &    0.13 &    0.13 &    0.14 \\
5 &    0.11 &    0.11 &    0.12 &    0.12 &    0.12 \\
6 &    0.14 &    0.18 &    0.26 &    0.28 &    0.28 \\
\bottomrule
\end{tabular}



In [212]:
### Generate std table 
table1_std = pd.DataFrame()
index = ['50','100','500','1000','1500']
for id_layer in range(1,8):
    layer = df[df.id_layer == id_layer]
    f = layer.groupby('n_neurons').std()['r2_max']
    table1_std = table1_std.append(f,ignore_index = True)
table1_std.to_pickle("table1_std.pkl")
print(table1_std.to_latex(float_format="{:0.2f}".format))

\begin{tabular}{lrrrrr}
\toprule
{} &  50.0   &  100.0  &  500.0  &  1000.0 &  1500.0 \\
\midrule
0 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
1 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
2 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
3 &    0.02 &    0.03 &    0.04 &    0.04 &    0.05 \\
4 &    0.08 &    0.08 &    0.08 &    0.08 &    0.08 \\
5 &    0.07 &    0.07 &    0.07 &    0.07 &    0.07 \\
6 &    0.15 &    0.15 &    0.13 &    0.13 &    0.12 \\
\bottomrule
\end{tabular}



In [213]:
### Generate stderr table
import numpy as np
table1_stderr = table1_std/np.sqrt(16)
table1_stderr.to_pickle("table1_stderr.pkl")
print(table1_stderr.to_latex(float_format="{:0.2f}".format))


\begin{tabular}{lrrrrr}
\toprule
{} &  50.0   &  100.0  &  500.0  &  1000.0 &  1500.0 \\
\midrule
0 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
1 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
2 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
3 &    0.01 &    0.01 &    0.01 &    0.01 &    0.01 \\
4 &    0.02 &    0.02 &    0.02 &    0.02 &    0.02 \\
5 &    0.02 &    0.02 &    0.02 &    0.02 &    0.02 \\
6 &    0.04 &    0.04 &    0.03 &    0.03 &    0.03 \\
\bottomrule
\end{tabular}



In [214]:
### Generate max table
table1_max = pd.DataFrame()
index = ['50','100','500','1000','1500']
for id_layer in range(1,8):
    layer = df[df.id_layer == id_layer]
    f = layer.groupby('n_neurons').max()['r2_max']
    table1_max = table1_max.append(f,ignore_index = True)
table1_max.to_pickle("table1_max.pkl")
print(table1_max.to_latex(float_format="{:0.2f}".format))


\begin{tabular}{lrrrrr}
\toprule
{} &  50.0   &  100.0  &  500.0  &  1000.0 &  1500.0 \\
\midrule
0 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
1 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
2 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
3 &    0.19 &    0.15 &    0.15 &    0.18 &    0.21 \\
4 &    0.27 &    0.31 &    0.33 &    0.34 &    0.31 \\
5 &    0.27 &    0.24 &    0.23 &    0.25 &    0.28 \\
6 &    0.43 &    0.48 &    0.50 &    0.53 &    0.48 \\
\bottomrule
\end{tabular}



In [215]:
### Generate min table
table1_min = pd.DataFrame()
index = ['50','100','500','1000','1500']
for id_layer in range(1,8):
    layer = df[df.id_layer == id_layer]
    f = layer.groupby('n_neurons').min()['r2_max']
    table1_min = table1_min.append(f,ignore_index = True)
table1_min.to_pickle("table1_min.pkl")
print(table1_min.to_latex(float_format="{:0.2f}".format))

\begin{tabular}{lrrrrr}
\toprule
{} &  50.0   &  100.0  &  500.0  &  1000.0 &  1500.0 \\
\midrule
0 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
1 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
2 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
3 &    0.00 &    0.00 &    0.00 &    0.00 &    0.00 \\
4 &    0.00 &    0.01 &    0.01 &    0.02 &    0.02 \\
5 &    0.01 &    0.02 &    0.01 &    0.01 &    0.01 \\
6 &    0.00 &    0.00 &    0.00 &    0.00 &    0.04 \\
\bottomrule
\end{tabular}



In [ ]:
### Generate table of mean(r2_max) accross folds 
table2 = pd.DataFrame()
index = ['1','2','3','4']
for id_layer in range(5,8):
    layer = df[df.id_layer == id_layer]
    f = layer.groupby('n_fold').mean()['r2_max']
    table2 = table2.append(f,ignore_index = True)
table2.to_pickle("table2.pkl")
print(table2.to_latex(float_format="{:0.2f}".format))

In [298]:
### Generate the number of the best fold per subject per layer
table3 = pd.DataFrame()
index = ['conv5', 'conv6', 'conv7']
df_alias = df[df.n_neurons == 1000]
a = []
for id_subject in range(1,18):
    if id_subject != 5 :
        arr_subject = []
        for id_layer in range(5,8):
            layer = df_alias[df_alias.id_layer == id_layer]
            subject = layer[layer.id_subject == id_subject][['r2_max']]
            best_fold = subject.to_numpy().argmax() + 1
            if best_fold == 0:
                print(id_layer, id_subject)
            arr_subject.append(best_fold)
        a.append(arr_subject)

In [299]:
a_np = np.array(a)
dump(a_np, 'best_fold.pkl')


['best_fold.pkl']

NameError: name 'df' is not defined

In [297]:
un, d, t,q

(16, 0, 6, 23)